In [1]:
using Makie

In [2]:
function LJforce(r2::Float64)
    return 12*(1-r2^3)/r2^7
end

function calforce(r,f,N)
  f .=0
  for i in 1:N-1, j in i+1:N
    r2 = (r[i,1]-r[j,1])^2 +(r[i,2]-r[j,2])^2 +(r[i,3]-r[j,3])^2     
    r2 = LJforce(r2)
    
    f[i,1] -= r2 * (r[j,1]-r[i,1]) 
    f[j,1] += r2 * (r[j,1]-r[i,1])
    f[i,2] -= r2 * (r[j,2]-r[i,2]) 
    f[j,2] += r2 * (r[j,2]-r[i,2]) 
    f[i,3] -= r2 * (r[j,3]-r[i,3]) 
    f[j,3] += r2 * (r[j,3]-r[i,3]) 
    
  end
  
end

calforce (generic function with 1 method)

In [3]:
function ルンゲクッタ(r::Array{Float64,2},N::Int64,
    rlist::Array{Float64,3},
    vlist::Array{Float64,3}; 
    T=20., dt=1e-3, repair = 50, Mcutoff=400, gamma = .99)
  
  v = Array{Float64,2}(undef,N,3)
  @. v = 0.
  #print(r)
  #dr1 =copy(r); dr2=copy(r); dr3=copy(r); dr4=copy(r)
  dv1 = Array{Float64,2}(undef,N,3)
  dv2 = Array{Float64,2}(undef,N,3)
  dv3 = Array{Float64,2}(undef,N,3)
  dv4 = Array{Float64,2}(undef,N,3)
  #dv1 =copy(r); dv2=copy(r); dv3=copy(r); dv4=copy(r)
  r1 = Array{Float64,2}(undef,N,3)
  r2 = Array{Float64,2}(undef,N,3)
  r3 = Array{Float64,2}(undef,N,3)
  r4 = Array{Float64,2}(undef,N,3)
  #r1 = copy(r); r2=copy(r); r3=copy(r); r4=copy(r)
  
  v1 = Array{Float64,2}(undef,N,3)
  v2 = Array{Float64,2}(undef,N,3)
  v3 = Array{Float64,2}(undef,N,3)
  v4 = Array{Float64,2}(undef,N,3)
  #v1 = copy(r); v2=copy(r); v3=copy(r); v4=copy(r)
  dr = Array{Float64,2}(undef,N,3)
  dv = Array{Float64,2}(undef,N,3)
  #dr = copy(r); dv=copy(r)
  #rlist[:,:,1] = r1
  #vlist[:,:,1] = v1
  
  for (i,t) in enumerate(0:dt:T)
    rlist[:,:,i] = r
    vlist[:,:,i] = v
    
    #dr1 = v
    calforce(r,dv1,N)
    
    #@. r1 = r + .5dt * dr1
    @. r1 = r + .5dt * v
    @. v1 = v + .5dt * dv1
    
    #dr2 = v1
    calforce(r1,dv2, N) 

    # k3
    #@. r2 = r + .5dt * dr2 
    @. r2 = r + .5dt * v1
    @. v2 = v + .5dt * dv2
    
    #dr3 = v2
    calforce(r2,dv3, N)

    # k4
    @. r3 = r + dt * v2 
    @. v3 = v + dt * dv3
    
    #dr4 = v3
    calforce(r3,dv4, N)

    @. dr = (dt/6.)*(v + 2v1 + 2v2 + v3) 
    @. dv = (dt/6.)*(dv1 + 2dv2 + 2dv3 + dv4)
    @. r += dr    
    @. v += dv 
    
  end
  
end


ルンゲクッタ (generic function with 1 method)

In [4]:
K = 4
L = 5
M = 5
N = K*L*M

r = Array{Float64,2}(undef,N,3)
I = 0
for i in 1:K
  for j in 1:L
    for k in 1:M
      I+=1
      r[I,1] = i*1.
      r[I,2] = j*1.
      r[I,3] = k*1.
  
    end
  end
end

N

100

In [5]:
dt = 1e-2
T = 20

numStep = length(0:dt:T)
rlist = Array{Float64,3}(undef,N,3,numStep)
vlist = Array{Float64,3}(undef,N,3,numStep)

r2 = copy(r)
# 天ぷら 天ぷら
@time ルンゲクッタ(r,N, rlist, vlist;T=T, dt=dt);
# 猪突ウ 猛進!!!
@time ルンゲクッタ(r2,N, rlist, vlist;T=T, dt=dt);

  6.709229 seconds (3.01 M allocations: 139.974 MiB, 2.05% gc time)
  5.302893 seconds (25 allocations: 37.922 KiB)


In [6]:
skip2 = 10
scene = Scene(resolution = (1000,1000))
mystep = Node(1)

scene = Makie.meshscatter!(
  scene, 
  lift(i -> rlist[:,1,i], mystep),
  lift(i -> rlist[:,2,i], mystep),
  lift(i -> rlist[:,3,i], mystep),
  markersize = .5,
  color = :pink,
  limits =FRect3D(Vec3f0(0.,0.,0.), Vec3f0(M+1,M+1,M+1))
 #limits = FRect(-2. , -2., 1.5M+4., 1.5M+4.)
  )
@time record(scene, "kaiteigo.mp4",1:skip2:numStep) do i
  #println(i)
  mystep[]=i
  end;

 63.322586 seconds (47.39 M allocations: 2.898 GiB, 2.19% gc time)
